<a href="https://colab.research.google.com/github/unizar-flav/KiPaD/blob/master/KiPaD_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kinetic Parameter Determination

In [ ]:
#@title Modules and functions
#@markdown You need to run this cell only once regardless of the number of datasets to be evaluated.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile # Necesary to compress the files into zip
import csv

from scipy.linalg import svd

from sklearn.linear_model import LinearRegression

from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, output_file, save, show

from bokeh.palettes import linear_palette, Viridis256
from bokeh.palettes import Category20
from bokeh.models import Button, CustomJS, TabPanel, Tabs, Legend, Span, Label
from bokeh.layouts import column
from bokeh.transform import linear_cmap
output_notebook()

from mpl_toolkits.mplot3d import Axes3D


from google.colab import files
from datetime import datetime
import os

!git clone https://github.com/Mario-uni/KiPaD.git
from KiPaD.funcionesGenerales import  procesa, argLeastSquares, deriv_RK


# Function lee_espectro
def lee_espectro(nombrFichs, skip_rows= 0):
  df_list = [] # Initialize an empty list to store DataFrames

  for fich in nombrFichs:
    #REead each file into a DataFrame
    temp_df = pd.read_csv(fich, skiprows=[skip_rows], index_col = 0)
    df_list.append(temp_df) # Append each DataFrame to the list

  # Concatenate all DataFrames into one
  df = pd.concat(df_list)

  # Sort the resulting DataFrame by index
  df = df.sort_index()

  # Let's get a name for the plots to follow which data was uploaded
  main = next((fich for fich in nombrFichs if "_t" in fich), None)
  return df, main


# Plot function
def create_spectra_plot(df, Title, x_axis, y_axis, Legend, width=1200, height=700):
    # Create a figure
    p = figure(title=Title,
               x_axis_label=x_axis,
               y_axis_label=y_axis,
               width=width, height=height)

    # Define font sizes for the title, axes, and labels
    p.title.text_font_size = '20pt'
    p.xaxis.axis_label_text_font_size = '16pt'
    p.yaxis.axis_label_text_font_size = '16pt'
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'

    # Generate a color palette using Viridis256
    n_lines = len(df.columns)
    colors = linear_palette(Viridis256, n_lines)


    indices = pd.to_numeric(df.index)
    # Plot each column as a line
    for idx, col in enumerate(df.columns):
        p.line(indices, df[col], legend_label=str(col), line_width=2, color=colors[idx])
        #p.line(pd.to_numeric(df.index), df[col], legend_label=str(col), line_width=2, color=colors[idx])
    # Customize the legend
    p.legend.title = Legend
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"  # Allows hiding lines by clicking their labels
    p.toolbar_location = "below"
    p.legend.visible = False # Initially hide the legend
    p.legend.label_text_font_size = '12pt'
    p.legend.title_text_font_size = '14pt'

    # Create a button to toggle the legend visibility
    button = Button(label="Toggle Legend", button_type= "success")

    # Custom JavaScript to toggle legend visibility
    button.js_on_click(CustomJS(args=dict(legend=p.legend[0]), code= """
    legend.visible = !legend.visible;  // Toggle the visibility
"""))

    # Return the plot object and button as a column layout
    return column(p,button)





# Scree Plot Method, with an elbow selection criterion based on the regression coefficient


def scree_plot_with_fit(singular_values, threshold):
    '''
    Plots scree plot of singular values and determine significant values using a linear fit.

    Parameters:
        singular_values (array-like): Array of singular values.
        threshold (float): Regression coefficient threshold (between 0 and 1) for linear fit.

    Returns:
        dict: Number of significant singular values and a Bokeh plot.
    '''

    n_values = len(singular_values)
    SSVs = 0  # Number of significant singular values to keep

    # Initialize Bokeh figure
    p = figure(title = " Scree Plot with Linear Fit",
               x_axis_label="Singular Value Index",
               y_axis_label= "Singular Values",
               width= 800, height=600)
    p.title.text_font_size = '20pt'
    p.xaxis.axis_label_text_font_size = '16pt'
    p.yaxis.axis_label_text_font_size = '16pt'
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'

    # Plot singular values
    indices = np.arange(1, n_values +1)
    p.scatter(indices, singular_values, size=8, color='blue', legend_label= "Singular Values")

    # Placeholder variables for the inear fit line data
    X_final, y_final_pred = None, None



    # Iterate through singular values, trying linear fits
    for i in range(2, n_values + 1):  # Start with at least two points for linear regression
        X = np.arange(1, i + 1).reshape(-1, 1)
        y = singular_values[:i]

        # Perform linear regression
        model = LinearRegression().fit(X, y)
        r_squared = model.score(X, y)  # Get the R^2 (regression coefficient)

        # If the fit quality falls below the threshold, stop
        if r_squared < threshold:
            SSVs = i - 1
            break
        else:
            SSVs = i
        # Update the final data for the significant linear fit
        X_final = np.arange(1, SSVs + 1).reshape(-1,1)
        y_final_pred = model.predict(X_final)

    # Plot the linear fit up to the last significant singular value
    p.line(X_final.flatten(), y_final_pred, line_width = 2, color= "red", line_dash= "dashed",
                           legend_label= "Linear Fit")

    # Add a vertical line to mark the cutoff for significant values
    cutoff_line = Span(location= SSVs, dimension= 'height', line_color="green", line_dash="dashed")
    p.add_layout(cutoff_line)

    # Add a label indicating the cutoff
    cutoff_label = Label(x=SSVs, y=singular_values[SSVs - 1], text = f'Significant Count = {SSVs}',
                         text_color='green', y_offset=10)
    p.add_layout(cutoff_label)

    # Customize Legend and toolbar
    p.legend.title = "Legend"
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    p.toolbar_location = "below"

    # Add a toogle button to control the legend visibility
    button = Button(label = "Toggle Legend", button_type = "success")
    button.js_on_click(CustomJS(args=dict(legend=p.legend[0]),
                                code="""
                                legend.visible = ! legend.visible;
                                """))

    #Show the plot with the toggle button
    plot=column(p,button)
    sol={'SSVs':SSVs, "plot":plot}
    return sol




def entropy_selection(singular_values, entropy_threshold):

    '''
    Entropy based method to determine the number of significant singular values.

    Parameters:
        singular_values (array-like): Array of singular values.
        threshold (float): PENDING
    Returns:
        dict: Number of significant singular values and a Bokeh plot.
    '''
    total_energy = np.sum(singular_values ** 2)

    # Calculate normalized singular values (f_j)
    f_j = singular_values ** 2 / total_energy

    # Calculate entropy
    entropy_val = np.sum(f_j * np.log(f_j)) / np.log(len(singular_values))

    # Uncomment the line below in order to check the entropy of the singular values:
    #print(f"\t Entropy of singular values: {entropy_val:.4f}")

    # Calculate cumulative entropy for each k
    cumulative_entropy = np.zeros(len(singular_values))
    for k in range(len(singular_values)):
        ff = f_j[:k+1]
        cumulative_entropy[k]=np.sum(ff*np.log(ff)/np.log(len(singular_values)))
    percentage=cumulative_entropy/entropy_val
    #print(percentage)

    # Find the smallest index k such that cumulative entropy meets the threshold
    significant_indices = np.where(percentage >= entropy_threshold)[0]

    if len(significant_indices) == 0:
        return 0  # Return 0 if no significant indices found
    else:
        return significant_indices[0] + 1  # Return the number of significant components




# Broken Stick Method
def broken_stick_method(singular_values):
    '''
    Broken Stick Method to determine the number of significant singular values.

    Parameters:
        singular_values (array-like): Array of singular values.

    Returns:
        dict: Number of significant singular values and a Bokeh plot.
    '''
    k = len(singular_values)

    # Calculate the broken stick values
    broken_stick = np.zeros(k)
    for i in range(1, k + 1):
        broken_stick[i - 1] = (1 / k) * np.sum([1 / j for j in range(i, k + 1)])

    # Normalize the singular values and broken stick values for comparison
    singular_values_normalized = singular_values / np.sum(singular_values)
    broken_stick_normalized = broken_stick / np.sum(broken_stick)

    # Initialize Bokeh figure
    p = figure(title="Broken Stick Model vs Singular Values",
               x_axis_label="Index",
               y_axis_label="Proportion of Variance",
               width=800, height=600)
    p.title.text_font_size = '20pt'
    p.xaxis.axis_label_text_font_size = '16pt'
    p.yaxis.axis_label_text_font_size = '16pt'
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'

    # Plot the normalized singular values
    indices = np.arange(1, k + 1)
    p.line(indices, singular_values_normalized, line_width=2, color="blue", legend_label="Singular Values")
    p.scatter(indices, singular_values_normalized, size=8, color="blue")

    # Plot the normalized broken stick values
    p.line(indices, broken_stick_normalized, line_width=2, line_dash="dashed", color="red", legend_label="Broken Stick")
    p.scatter(indices, broken_stick_normalized, size=8, color="red")

    # Customize legend and toolbar
    p.legend.title = "Legend"
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    p.toolbar_location = "below"

    # Determine the number of significant singular values using the broken stick rule
    SSVs = np.where(singular_values_normalized > broken_stick_normalized)[0][-1] + 1

    SSVs = 0
    for i in range(k):
      if singular_values_normalized[i]> broken_stick_normalized[i]:
        SSVs += 1
      else:
        break

    # Add a toggle button to control the legend visibility
    button = Button(label="Toggle Legend", button_type="success")
    button.js_on_click(CustomJS(args=dict(legend=p.legend[0]), code="""
        legend.visible = !legend.visible;
    """))

    #Show the plot with the toggle button
    plot=column(p,button)
    sol={'SSVs':SSVs, "plot":plot}
    return sol



# Function for matrix approximation from the number of SSV selected:
def matrix_approximation(A, n):
    """
    Approximates matrix A using the top n singular values.

    Parameters:
    - A: The original matrix to approximate.
    - n: Number of significant singular values to use for approximation.

    Returns:
    - A_approx: The approximated matrix.
    """
    # Perform SVD using scipy.linalg.svd
    U, Sigma, VT = svd(A, full_matrices=False)

    # Truncate the matrices to keep only the top 'n' singular values
    U_n = U[:, :n]             # Keep the first 'n' columns of U
    Sigma_n = np.diag(Sigma[:n])  # Keep the first 'n' singular values (diagonal matrix)
    VT_n = VT[:n, :]           # Keep the first 'n' rows of V^T

    # Compute the approximated matrix
    A_approx = np.dot(U_n, np.dot(Sigma_n, VT_n))  # A_approx = U_n * Sigma_n * VT_n

    return A_approx




# Functions for the model


 # Name pending
def create_ode_matrix(n_species, k_vals):
    """
    Creates the ODE matrix to represent a system of species with the specified rate constants.

    Parameters:
        n_species (int): Number of species in the system.
        params (dict): Dictionary of rate constants, e.g., {'k1': value, 'k_1': value, 'k2': value, ...}.

    Returns:
        np.ndarray: Matrix that aligns with the ODEs specified for each species.
    """
    # Initialize an n_species x n_species matrix with zeros
    ode_matrix = np.zeros((n_species, n_species))

    # Populate the ODE matrix according to the specified rules
    for i in range(n_species):
        # Rate constant for reaction from species i to species i+1, if within bounds
        if i + 1 < n_species:
            ode_matrix[i, i] -= k_vals.get(f'k{i+1}', 0)      # Outflow from species i to i+1
            ode_matrix[i + 1, i] += k_vals.get(f'k{i+1}', 0)  # Inflow to species i+1 from i

        # Rate constant for reaction from species i+1 back to species i, if within bounds
        if i - 1 >= 0:
            ode_matrix[i, i] -= k_vals.get(f'k_{i}', 0)       # Outflow from species i to i-1
            ode_matrix[i - 1, i] += k_vals.get(f'k_{i}', 0)   # Inflow to species i-1 from i

    return ode_matrix





def deriv_conc(conc,t, ks_matrix):
  '''
  Calculate the concentration derivative of every species .
  The system of ODE's characterizing the reaction model is passed as a matrix with the rate constants
  as coefficients. (REVISE)

  Parameters
      conc: Array con las concentraciones de las especies.
      t : times points at which to calculate de derivative of the concentration with respect time.
      params: a dictionary that contains the rate constants.

  Returns:
      np.ndarray: vector that contains the derivative of the concentrations of each species
                  at the specified time point t. the rate of change of conecntration for each
                  species in the system at the given time t.

  '''
  return np.dot(ks_matrix, conc)




# Algoritmo Runge-Kutta 4º orden
def deriv_RK (fDeriv,x,t,deltaT,paramDeriv):
  #print(f"paramDeriv is: {paramDeriv}")
  k1 = fDeriv ( x, t, paramDeriv)
  k2 = fDeriv ( x+k1*deltaT/2, t+ deltaT/2, paramDeriv)
  k3 = fDeriv ( x+k2*deltaT/2, t+ deltaT/2, paramDeriv)
  k4 = fDeriv ( x+k3*deltaT, t+ deltaT, paramDeriv)

  return (k1+2*k2+2*k3+k4)/6

def solv_conc_profile (k_vals, f_deriv, Conc_0, t):
  """
  Solves the concentration profile of the reaction over time using
  a 4th-order Runge-Kutta (RK4) method, allowing for variable time steps.

  Parameters:
  - f: Function that computes the derivative (reaction model)
  - y0: Initial concentrations of the species
  - t: Array or list of time points (can have non-uniform intervals)
  - k_vals: Dictionary of reaction kinetic constants needed for the reaction model

  Returns:
  - df: DataFrame containing the cncentration profile for each species over time
  """

  # Extract Conc_0 from 'initial_conc' in params
  initial_conc = np.array(list(Conc_0.values()))


  n_steps=len(t)
  n_species = len(initial_conc)


  # Initialize the solution array to store each species' concentration at each time step
  solution = np.zeros((n_steps, n_species))
  #print(solution[0])
  solution[0]= initial_conc # Initial conditions

  # We create the ODE system as matrix with the rate constants dispossed as its coefficients
  MCoef= create_ode_matrix(n_species, k_vals)



  # Iterate through each time step using the function deriv_RK from funcionesGenerales
  for i in range(1, n_steps):
    current_t = t[i-1]
    next_t =t[i]
    current_y = solution[i-1]

    # Here, calculate the time intercal (delta_t) dynamically
    delta_t = next_t - current_t


    # Use deriv_RK to calculate the next step, passing `f_deriv` as the first argument
    solution[i] = current_y + delta_t * deriv_RK(
        f_deriv, current_y, current_t, delta_t, MCoef
    )

  # df = pd.DataFrame(solution, index=t, columns= ["A", "B", "C", "D"]) # shape (time, species)
  # Generate column names based on the number of species
  column_names = [f"{chr(65 + j)}" for j in range(n_species)]  # 'A', 'B', 'C', ...

  # Create the DataFrame without empty columns
  df = pd.DataFrame(solution, index=t, columns=column_names)  # shape (time, species)
  return df

#concentration_matrix_rk4 = solv_conc_profile(initial_ks, deriv_conc, initial_conc, t=df.index)

# Output the concentration matrix
#concentration_matrix_rk4

#def solv_conc_profile (params, f_deriv, Conc_0, t):
def species_spectra (k_vals, f_deriv, Conc_0, t, abs, pathlength, method):
  initial_conc = np.array(list(Conc_0.values()))


  n_species = len(initial_conc)
  # Extract the reaction model (fDeriv)
  #model = k_vals.get('fDeriv')

  C_profile = solv_conc_profile(k_vals, f_deriv, Conc_0,t)

  #print(type(abs))
  spectra = {}
  if n_species == 2:
    spectra['A'] = abs.iloc[0,:]/(C_profile.iloc[0,0] * pathlength)
    spectra['B'] = abs.iloc[-1,:]/(C_profile.iloc[-1,1] * pathlength)
  elif n_species == 3:
    spectra['A'] = abs.iloc[0,:]/(C_profile.iloc[0,0] * pathlength )
    spectra['B'] = None
    spectra['C'] = abs.iloc[-1,:]/(C_profile.iloc[-1,2] * pathlength )


    max_conc_B = C_profile.iloc[:,1].abs().max() # absolute maximum
    #print(max_conc_B)
    tB_label = C_profile.iloc[:,1].abs().idxmax() # This get the time not the positional idx
    tB = C_profile.index.get_loc(tB_label)
    #print(tB)
    spectra['B'] = (1/(max_conc_B ))*(abs.iloc[tB,:]/(pathlength) -
                                      spectra['A']*C_profile.iloc[tB,0]  -
                                      spectra['C']*C_profile.iloc[tB,-1] )
  elif n_species == 4:
    spectra['A'] = abs.iloc[0,:]/(C_profile.iloc[0,0] * pathlength)
    spectra['B'] = None
    spectra['C'] = None
    spectra['D'] = abs.iloc[-1,:]/(C_profile.iloc[-1,3] * pathlength)

    max_conc_B = C_profile.iloc[:,1].abs().max() # absolute maximum
    #print(max_conc_B)
    tB_label = C_profile.iloc[:,1].abs().idxmax() # This get the time not the positional idx
    tB = C_profile.index.get_loc(tB_label)
    #print(tB)

    max_conc_C = C_profile.iloc[:,2].abs().max() # absolute maximum

    tC_label = C_profile.iloc[:,2].abs().idxmax() # This get the time not the positional idx
    tC = C_profile.index.get_loc(tC_label)

    factor = C_profile.iloc[tC,2]/(C_profile.iloc[tC,2]*C_profile.iloc[tB,1] -
                                   C_profile.iloc[tB,2]*C_profile.iloc[tC,1])
    mu = C_profile.iloc[tB,2]/C_profile.iloc[tC,2]

    spectra['B']= factor *(abs.iloc[tB,:]/(pathlength) -
                           spectra['A']*C_profile.iloc[tB,0] -
                           spectra['D']*C_profile.iloc[tB,-1] -
                           mu*(abs.iloc[tC,:]/(pathlength) -
                               spectra['A']*C_profile.iloc[tC,0] -
                               spectra['D']*C_profile.iloc[tC,-1]))
    spectra['C'] =(1/C_profile.iloc[tC,2])* (abs.iloc[tC,:]/(pathlength) -
                                        spectra['A']*C_profile.iloc[tC,0] -
                                        spectra['B']*C_profile.iloc[tC,1] -
                                        spectra['D']*C_profile.iloc[tC,-1] )
  sol = pd.DataFrame.from_dict(spectra)
  sol= sol.T

  # Until here is the calculation of the spectra explicitly with assumptions

  # Now we are going to use the implicit approach of the explicit approach above

  if method =="Explicit":
    result=sol
  elif method == "Implicit":
     # Generalized code to find max value and corresponding index
    max_indices = {}
    for col in C_profile.columns:
      # max_value = C_profile[col].max()
      max_index = C_profile[col].idxmax() # Get the index of the max value
      max_indices [col] = max_index

    # Use the indices found to slice the DataFrames
    indices =list(max_indices.values())
    reduced_conc=C_profile.loc[indices]
    reduced_abs =abs.loc[indices]
    # Solve the system of equations C^-1*A = E
    sol_imp = np.dot(np.linalg.inv(reduced_conc), reduced_abs)

    # Assign alphabetical names to the indices (A, B, C, ...)
    alphabet_indices = [chr(65 + i) for i in range(len(indices))]  # 65 is ASCII for 'A'
    sol_imp=pd.DataFrame(sol_imp, index=alphabet_indices, columns=abs.columns)
    sol_imp=sol_imp
    result=sol_imp
  elif method == "Pseudo-inverse":
    # Now we are going to use the pseudoinverse of the concentration to estimate the spectroscopic species
    # (extinction coefficients)
    sol_ps=np.dot(np.linalg.pinv(C_profile),datos_approx_df)
    alphabet_indices = [chr(65 + i) for i in range(len(C_profile.columns))]  # 65 is ASCII for 'A'
    sol_ps=pd.DataFrame(sol_ps, index=alphabet_indices, columns=abs.columns)
    sol_ps=sol_ps
    result=sol_ps
  else:
    print("Input for method not valid")


  #return sol, sol_imp, sol_ps
  return result
#def species_spectra (params, f_deriv, Conc_0, t, abs, pathlength):






def Model_spectra(k_vals,f_deriv, Conc_0, t, abs, pathlength, original_data, method, fitting = True):

  n_species = len(np.array(list(Conc_0.values())))
  #Conc_0= Conc_0 # with the "*" operator we unpack the values of the dictionary

  #Solve for concentrations
  C_matrix=solv_conc_profile(k_vals, f_deriv, Conc_0,t)
  #print(f'C_matrix{type(C_matrix)}')

  #Construct full extinction coefficient matrix
  S_matrix= species_spectra (k_vals, f_deriv, Conc_0, t, abs, pathlength, method)
  #print(f'S_matrix{type(S_matrix)}')
  #print("Shape of C_matrix:", C_matrix.shape)  # Concentration matrix
  #print("Shape of S_matrix:", S_matrix.shape)  # Extinction coefficient matrix

  #Use Lambert-Beer Law to calculate predicted absorbance (n_Lambda x n_t)
  D_model = pathlength * np.dot(C_matrix, S_matrix)

  D_exp = abs
  D_org = original_data
  #print("Shape of D_exp:", D_exp.shape)  # Experimental absorbance
  #print("Shape of D_model:", D_model.shape)  # Predicted absorbance

  #Compute residuals (difference between experimental abd predicted absorbance)
  residuals_denoised= D_exp - D_model
  residuals= D_org - D_model
  #print(f'residuals{type(residuals)}')
  D_model_df =pd.DataFrame(D_model, index=D_exp.index, columns= D_exp.columns)

  if fitting:
    sol= D_model_df.values.flatten()
  else:
    sol = {
    "params": k_vals,
    "Conc_0": Conc_0,
    "pathlength": pathlength,
    "n_species": n_species,
    "D_orig": original_data,
    "D_approx": abs,
    "D_model": D_model_df,
    "C_matrix": C_matrix,
    "S_matrix": S_matrix,
    "residuals": residuals,
    "residuals_denoised": residuals_denoised
}
  #return D_model_df
  return sol



In [ ]:
#@title Upload files

#@markdown Here you have the option to upload different combinations of files as mentioned in the README.


# Upload the file and save in a dictionary
uploaded=files.upload()

# Obtain the uploaded file name from the dictionary


file_names = list(uploaded.keys())



datos, file_name= lee_espectro(file_names)
#datos = pd.read_csv(file_name,skiprows=[0])


datos

In [ ]:
#@title Spectra plot
# @markdown Plots Absorbance vs Wavelength and Absorbance vs Time.

df=datos
df_transposed = df.T # Rows to columns and columns to rows

wavelength_plot_2D = create_spectra_plot(df_transposed, Title = f"Absorbance vs Wavelength // {file_name}",
                                         x_axis="Wavelength (nm)", y_axis ="Absorbance",
                                         Legend = "Time (s)")

time_plot_2D = create_spectra_plot(df, Title= f"Absorbance vs Time // {file_name}",
                                   x_axis = "Time (s)", y_axis = "Absorbance", Legend = "Wavelength (nm)" )

plots = [wavelength_plot_2D,time_plot_2D]


# Create tabs to display both plots
wavelength_panel = TabPanel(child=wavelength_plot_2D, title="Wavelength Plot")
time_panel = TabPanel(child=time_plot_2D, title="Time Plot")

tabs = Tabs(tabs=[wavelength_panel, time_panel])

# Show the plots
show(tabs)

In [ ]:
#@title Singular Value Determination (SVD) and Identification of the Significant Singular Values (SSV)
# Now we need a method to determine the number of significant singular values

#@markdown **SVD Calculation**
#@markdown
#@markdown Only check the box below if you want to visualize the Singular Values (Sigma),
#@markdown the transpose matrix (U) and the right matrix (VT).
Check_SVD = False #@param {type:"boolean"}

#@markdown ****
#@markdown **SSV Determination**
scree_plot_th = 0.9 #@param{type: "number"}

entropy_threshold= 0.9 #@param {type: "number"}
if entropy_threshold <0 or entropy_threshold>1:
  entropy_threshold=0.85
  print("Value inputted not valid, entropy_threshold has taken the preset value of 0.85")


# Convert the dataframe into a Numpy array for the svd to work
datos_array= datos.to_numpy()
datos_array
# Save the columns and row names
Times= datos.index
Wavelengths=datos.columns



# Performn SVD
U, Sigma, Vt = svd(datos_array, full_matrices= False)


# Display results

U_df = pd.DataFrame(U)

Sigma_df = pd.DataFrame(Sigma)

Vt_df = pd.DataFrame(Vt)
if Check_SVD:
  print("U Matrix:\n",U_df)
  print("Singular Values:\n",Sigma_df)
  print("V Transpose Matrix:\n",Vt_df)
else:
  pass







# Determine the number of significant singular values using the scree plot method
n_significant_scree_plot = scree_plot_with_fit(Sigma, scree_plot_th)
print(f"Number of significant singular values (Scree Plot Method): {n_significant_scree_plot['SSVs']}")

# Determine the number of significant singular values using entropy-based selection
n_significant_entropy = entropy_selection(Sigma, entropy_threshold)
print(f"\nNumber of significant singular values (Entropy Method): {n_significant_entropy}")

# Determine the number of significant singular values using the broken stick method
n_significant_broken_stick = broken_stick_method(Sigma)
print(f"\nNumber of significant singular values (Broken Stick Method): {n_significant_broken_stick['SSVs']} \n")


plots = [n_significant_scree_plot['plot'], n_significant_broken_stick['plot']]


# Create tabs to display both plots
scree_plot_panel = TabPanel(child=n_significant_scree_plot['plot'], title="Scree Plot")
broken_stick_panel = TabPanel(child=n_significant_broken_stick['plot'], title="Broken Stick Plot")

tabs = Tabs(tabs=[scree_plot_panel, broken_stick_panel])

# Show the plots
show(tabs)


In [ ]:
#@title Dimensionality reduction and Matrix Approximation

#@markdown **Number of Significant Singular Values (SSVs):**
SSVs = 3 #@param {type:"number"}
#SSV = n_significant_entropy #@param [n_significant_entropy, n_significant_broken, n_significant_manual]



datos_approx = matrix_approximation(datos_array, SSVs)
#print(datos_approx.shape)
#print(datos_array.shape)
print(f"Approximation of the original data, using the first {SSVs} SSVs: \n")
datos_approx_df= pd.DataFrame(datos_approx, index=Times, columns= Wavelengths)
datos_approx_df

In [ ]:
#@title Spectra plot
# @markdown Plots Absorbance vs Wavelength and Absorbance vs Time.

df=datos_approx_df
df_transposed = df.T # Rows to columns and columns to rows

wavelength_plot_2D = create_spectra_plot(df_transposed, Title = f"Absorbance vs Wavelength // {file_name}",
                                         x_axis="Wavelength (nm)", y_axis ="Absorbance",
                                         Legend = "Time (s)")

time_plot_2D = create_spectra_plot(df, Title= f"Absorbance vs Time // {file_name}",
                                   x_axis = "Time (s)", y_axis = "Absorbance", Legend = "Wavelength (nm)" )

plots = [wavelength_plot_2D,time_plot_2D]


# Create tabs to display both plots
wavelength_panel = TabPanel(child=wavelength_plot_2D, title="Wavelength Plot")
time_panel = TabPanel(child=time_plot_2D, title="Time Plot")

tabs = Tabs(tabs=[wavelength_panel, time_panel])

# Show the plots
show(tabs)

In [ ]:
#@title Reaction Model Parameters { run: "auto" }

#@markdown **Number of species:**
n_species = 3 #@param{type: "slider", min:2, max:4, step:1}

#@markdown **Pathlength of the cuvette (cm):**
pathlength = 1 #@param {type:"number"}

#@markdown ****
# Initial concentrations of species
#@markdown **Initial Concentrations (μM):**
A0 = 0.0 #@param {type:"number"}

B0 = 0.0 #@param {type:"number"}

C0 = 0.0 #@param {type:"number"}

D0 = 0.0 #@param {type:"number"}

#@markdown ****

# Enter the rate constants for the reactions
#@markdown **Kinetic Rates (1/s):**
k1 = 0 #@param {type:"number"}
k1_fixed = True  # @param {type: "boolean"}
#@markdown
k_1 = 0 #@param {type:"number"}
k_1_fixed = True  # @param {type: "boolean"}
#@markdown
k2 = 0 #@param {type:"number"}
k2_fixed = True  # @param {type: "boolean"}
#@markdown
k_2 = 0 #@param {type:"number"}
k_2_fixed = True  # @param {type: "boolean"}
#@markdown
k3 = 0 #@param {type:"number"}
k3_fixed = True  # @param {type: "boolean"}
#@markdown
k_3 = 0 #@param {type:"number"}
k_3_fixed = True  # @param {type: "boolean"}

#@markdown ****




# Define rate constants and their fixed status in a dictionary
rate_constants_data = {
    'k1': (k1, k1_fixed),
    'k_1': (k_1, k_1_fixed),
    'k2': (k2, k2_fixed),
    'k_2': (k_2, k_2_fixed),
    'k3': (k3, k3_fixed),
    'k_3': (k_3, k_3_fixed)
}

# Initialize dictionaries for fixed and variable rate constants (these last ones are to be optimized later)
fixed_ks = {}
variable_ks = {}

# Loop over each rate constant and classify it into fixed or variable
for rate, (value, is_fixed) in rate_constants_data.items():
    if is_fixed:
        fixed_ks[rate] = value
    else:
        variable_ks[rate] = value

# Output for debugging or checking
#print("Fixed Rate Constants:", fixed_ks)
#print("Variable Rate Constants:", variable_ks)




# Define initial concentrations and their fixed status in a dictionary
concentration_data= {
    'A0': A0,
    'B0': B0,
    'C0': C0,
    'D0': D0
}

# Slice the dictionary using a subset of its keys
species_list = ['A0', 'B0', 'C0', 'D0'][:n_species]
# Here we get the sliced dictionary
initial_conc = {key:concentration_data[key] for key in species_list}

#

# Output for debugging or checking
#print("Fixed Concentrations:", fixed_conc)
#print("Variable Concentrations:", variable_conc)


# Output of the fixed and variable rate constants
print("Fixed Rate Constants:")
for key, value in fixed_ks.items():
    print(f"{key} = {value}")

print("\nVariable Rate Constants:")
for key, value in variable_ks.items():
    print(f"{key} = {value}")

print("\nInitial Concentrations:")
for key, value in initial_conc.items():
    print(f"{key} = {value}")

# Group both rate constant dictionaries into one
initial_ks = {**fixed_ks, **variable_ks}

In [ ]:
#@title Procesa

#@markdown Method use for estimating the spectroscopic species:
Method="Pseudo-inverse" #@param["Pseudo-inverse", "Explicit", "Implicit"]

#@markdown The parameter above heavily influences the goodness of fitting.
#@markdown Select **"Pseudo-inverse"** for the best fitting, however, it requires a reasonable
#@markdown first estimation of the rate constant. Otherwise, select **"Explicit"** to obtain an initial
#@markdown idea of the possible magnitude of the rate constants.
initial_params= {**initial_ks }
initial_params_var= {**variable_ks}


#initial_params['n_species']=n_species # This dictionary contains the rate constants and the number of species
#initial_params['pathlength']= pathlength # We add pathlength to the dictionary to streamline the parameters onto
# a single dictionary

# Prepare the list of the parameters names to be opimized
nombrParVar = list(initial_params_var.keys())

# Independent values (time and wavelength) and dependent values (Absorbance)
fKwargs = dict(t=datos_approx_df.index.values,
               f_deriv=deriv_conc,
               Conc_0=initial_conc,
               abs=datos_approx_df,
               pathlength=pathlength,
               original_data= datos,
               method= Method,
               fitting=True
                )

abs= datos_approx_df


#cotaInf = [0 for param in nombrParVar]
#cotaSup= [2 if param =='Ro' else np.inf for param in nombrParVar]

sol=procesa(argLeastSquares = argLeastSquares,
            dictParEstim = initial_params,
            nombrParVar = nombrParVar,
            f = Model_spectra,
            fKwargs = fKwargs,
            Y = datos_approx_df.values.flatten()
            )

In [ ]:
#@title Model's plots
# @markdown This cell create plots with the information obtained by the model and
#@markdown the optimization process.

ad_parameters= sol['parAjustados']
Model= Model_spectra( ad_parameters, deriv_conc, initial_conc,datos_approx_df.index, datos_approx_df, pathlength,datos, method= Method, fitting=False,)
df_e=Model['D_model']


df_e_transposed = df_e.T # Rows to columns and columns to rows

wavelength_plot_2D = create_spectra_plot(df_e_transposed, Title = f"Absorbance vs Wavelength // {file_name}",
                                         x_axis="Wavelength (nm)", y_axis ="Absorbance",
                                         Legend = "Time (s)")

time_plot_2D = create_spectra_plot(df_e, Title= f"Absorbance vs Time // {file_name}",
                                   x_axis = "Time (s)", y_axis = "Absorbance", Legend = "Wavelength (nm)" )




df_conc= Model['C_matrix']
df_conc

conc_profile_plot = create_spectra_plot(df_conc, Title = f"Concentration over time // {file_name}",
                                         x_axis="Time (s)", y_axis ="Concentration (μM)",
                                         Legend = "Species")
#show(conc_profile_plot)

df_spectra=Model['S_matrix'].T
df_spectra


spectra_species_plot = create_spectra_plot(df_spectra, Title = f"Spectroscopic species // {file_name}",
                                         x_axis="Wavelength (nm)", y_axis ="Extinction Coefficient (1/(μM*cm))",
                                         Legend = "Species")
#show(spectra_profile_plot)

# Original - Model
df_res=Model['residuals']
df_res_transposed =df_res.T

wavelength_res_plot_2D = create_spectra_plot(df_res_transposed, Title = f"Absorbance vs Wavelength // {file_name}",
                                         x_axis="Wavelength (nm)", y_axis ="Absorbance",
                                         Legend = "Time (s)")
time_res_plot_2D = create_spectra_plot(df_res, Title= f"Absorbance vs Time // {file_name}",
                                   x_axis = "Time (s)", y_axis = "Absorbance", Legend = "Wavelength (nm)" )

# Denoised - Model
df_res_d=Model['residuals_denoised']
df_res_d_transposed =df_res_d.T

wavelength_res_d_plot_2D = create_spectra_plot(df_res_d_transposed, Title = f"Absorbance vs Wavelength // {file_name}",
                                         x_axis="Wavelength (nm)", y_axis ="Absorbance",
                                         Legend = "Time (s)")
time_res_d_plot_2D = create_spectra_plot(df_res_d, Title= f"Absorbance vs Time // {file_name}",
                                   x_axis = "Time (s)", y_axis = "Absorbance", Legend = "Wavelength (nm)" )


plots = [wavelength_plot_2D,wavelength_res_plot_2D,time_plot_2D,time_res_plot_2D, conc_profile_plot, spectra_species_plot,wavelength_res_d_plot_2D, time_res_d_plot_2D ]


# Create tabs to display both plots
wavelength_panel = TabPanel(child=wavelength_plot_2D, title="Wavelength Plot")
wavelength_res_panel = TabPanel(child=wavelength_res_plot_2D, title="Wavelength Residuals Plot")
wavelength_res_d_panel = TabPanel(child=wavelength_res_d_plot_2D, title="Wavelength Residuals Denoised Plot")

time_panel = TabPanel(child=time_plot_2D, title="Time Plot")
time_res_panel = TabPanel(child=time_res_plot_2D, title="Time Residuals Plot")
time_res_d_panel = TabPanel(child=time_res_d_plot_2D, title="Time Residuals Denoised Plot")

conc_profile_panel= TabPanel(child=conc_profile_plot, title="Concentration Profile")
spectra_species_panel = TabPanel(child= spectra_species_plot, title= "Spectroscopic Species")

tabs = Tabs(tabs=[wavelength_panel, time_panel, conc_profile_panel, spectra_species_panel, wavelength_res_panel,wavelength_res_d_panel, time_res_panel, time_res_d_panel ])

# Show the plots
show(tabs)

In [ ]:
#@title Export results

# @markdown Write the name for the zip file that contains the inputted and produced data.

# Initial experimental data
Model['D_orig'].to_csv('Original_experimental_data.csv', index=True)

# Denoised experimental data
Model['D_approx'].to_csv('Denoised_experimental_data.csv', index=True)

# Modeled data
Model['D_model'].to_csv('Modeled_data.csv', index=True)

# Residuals from Original - Modeled data
Model['residuals'].to_csv('Residuals_OrigMod.csv', index=True)

# Residuals from Denoised - Modeled data
Model['residuals_denoised'].to_csv('Residuals_DenMod.csv', index=True)

# Concentration profile
Model['C_matrix'].to_csv('Concentration_profile.csv', index=True)

# Spectroscopic species
Model['S_matrix'].to_csv('Spectroscopic_species.csv', index=True)



# Create a CSV
with open('Fitting_result.csv', mode='w', newline='') as file:
  writer = csv.writer(file)

  #Add blank rows
  writer.writerow([''] * 7)
  writer.writerow([''] * 7)

  # n_species & pathlength Section
  writer.writerow(['n_species', n_species])
  writer.writerow(['pathlength (cm)', pathlength])
  writer.writerow([''] * 7)

  # Initial concentrations Section
  writer.writerow(['INITIAL CONCENTRATIONS:'])
  writer.writerow(['A0', Model['Conc_0']['A0']
                   if 'AO' in Model['Conc_0'] else ''])
  writer.writerow(['B0', Model['Conc_0']['B0']
                   if 'BO' in Model['Conc_0'] else ''])
  writer.writerow(['C0', Model['Conc_0']['C0']
                   if 'CO' in Model['Conc_0'] else ''])
  writer.writerow(['D0', Model['Conc_0']['D0']
                   if 'DO' in Model['Conc_0'] else ''])

  # Headers for the rate constants results
  writer.writerow(['INITIAL ks', '', 'ADJUSTED ks','', 'STD ks'] )
  #k1 row
  writer.writerow(['k1', initial_ks['k1'], '', 'k1', sol['parAjustados']['k1'], '',
        'k1_std', sol['sdPar'].get('k1_std', '') if 'k1' in variable_ks else ''])  # Display k1_std only if it is a variable parameter
  #k_1 row
  writer.writerow(['k_1', initial_ks['k_1'], '', 'k_1', sol['parAjustados']['k_1'], '',
        'k_1_std', sol['sdPar'].get('k_1_std', '') if 'k_1' in variable_ks else ''])  # Display k_1_std only if it is a variable parameter

  #k2 row
  writer.writerow(['k2', initial_ks['k2'], '', 'k2', sol['parAjustados']['k2'], '',
        'k2_std', sol['sdPar'].get('k2_std', '') if 'k2' in variable_ks else ''])  # Display k2_std only if it is a variable parameter
  #k_2 row
  writer.writerow(['k_2', initial_ks['k_2'], '', 'k_2', sol['parAjustados']['k_2'], '',
        'k_2_std', sol['sdPar'].get('k_2_std', '') if 'k_2' in variable_ks else ''])  # Display k_2_std only if it is a variable parameter

  #k3 row
  writer.writerow(['k3', initial_ks['k3'], '', 'k3', sol['parAjustados']['k3'], '',
        'k3_std', sol['sdPar'].get('k3_std', '') if 'k3' in variable_ks else ''])  # Display k3_std only if it is a variable parameter
  #k_3 row
  writer.writerow(['k_3', initial_ks['k_3'], '', 'k_3', sol['parAjustados']['k_3'], '',
        'k_3_std', sol['sdPar'].get('k_3_std', '') if 'k_3' in variable_ks else ''])  # Display k_3_std only if it is a variable parameter


  # Add blank rows
  writer.writerow([''] * 7)
  writer.writerow([''] * 7)

  # R2 Section
  writer.writerow(['R2'])
  writer.writerow(['R2', sol['R2']])
  writer.writerow([''] * 7)
  writer.writerow([''] * 7)

  # Detalles Section
  writer.writerow(['Details'])

  # x values (convert floats to strings to concatenate with 'x')
  writer.writerow(['x'] + [str(x) for x in sol['detalles']['x']])

  # cost
  writer.writerow(['cost', sol['detalles']['cost']])

  # Add blank rows
  writer.writerow([''] * 7)

  # grad values (convert floats to strings)
  writer.writerow(['grad'] + [str(g) for g in sol['detalles']['grad']])

  # optimality
  writer.writerow(['optimality', sol['detalles']['optimality']])

  # active_mask (convert integers to strings)
  writer.writerow(['active_mask'] + [str(a) for a in sol['detalles']['active_mask']])

  # nfev, njev, status, message, success
  writer.writerow(['nfev', sol['detalles']['nfev']])
  writer.writerow(['njev', sol['detalles']['njev']])
  writer.writerow(['status', sol['detalles']['status']])
  writer.writerow(['message', sol['detalles']['message']])
  writer.writerow(['success', sol['detalles']['success']])

  # fun values (split into multiple rows)
  fun_values = sol['detalles']['fun']
  for row in fun_values:
    writer.writerow(['fun'] + [str(row)])

  # Add blank rows
  writer.writerow([''] * 7)

  # jac values (split into multiple rows)
  for row in sol['detalles']['jac']:
      writer.writerow(['jac'] + [str(v) for v in row])

  # Add blank rows
  writer.writerow([''] * 7)





# Then we proceed to save all the files and compressed them into a zip file

# Take the current date and hour ()
current_time = datetime.now().strftime("%d%m%Y%H%M%S")

# Define the prefix and create the complete name of the zip file
name ="spectra_" #@param {type: "string"}
zip_filename = f"{name}{current_time}.zip"

# Create a zip file with the name written
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # Add CSV files
    zipf.write('Original_experimental_data.csv')
    zipf.write('Denoised_experimental_data.csv')
    zipf.write('Modeled_data.csv')
    zipf.write ('Residuals_OrigMod.csv')
    zipf.write ('Residuals_DenMod.csv')
    zipf.write ('Concentration_profile.csv')
    zipf.write ('Spectroscopic_species.csv')
    zipf.write ('Fitting_result.csv')



# Download the zipped file
files.download(zip_filename)